In [ ]:
#read a h5ad file
"C:\Users\ellio\Downloads\scRNA_BCR_TCR.h5seurat"

#read a h5ad file



In [2]:
# Import required libraries
import scanpy as sc
import pandas as pd

# Read the h5ad file
adata = sc.read_h5ad("C:/Users/ellio/Downloads/scRNA_BCR_TCR.h5ad")



In [9]:
adata



AnnData object with n_obs × n_vars = 49910 × 36601
    obs: 'sample', 'has_ir', 'IR_VJ_1_c_call', 'IR_VJ_2_c_call', 'IR_VDJ_1_c_call', 'IR_VDJ_2_c_call', 'IR_VJ_1_consensus_count', 'IR_VJ_2_consensus_count', 'IR_VDJ_1_consensus_count', 'IR_VDJ_2_consensus_count', 'IR_VJ_1_d_call', 'IR_VJ_2_d_call', 'IR_VDJ_1_d_call', 'IR_VDJ_2_d_call', 'IR_VJ_1_duplicate_count', 'IR_VJ_2_duplicate_count', 'IR_VDJ_1_duplicate_count', 'IR_VDJ_2_duplicate_count', 'IR_VJ_1_j_call', 'IR_VJ_2_j_call', 'IR_VDJ_1_j_call', 'IR_VDJ_2_j_call', 'IR_VJ_1_junction', 'IR_VJ_2_junction', 'IR_VDJ_1_junction', 'IR_VDJ_2_junction', 'IR_VJ_1_junction_aa', 'IR_VJ_2_junction_aa', 'IR_VDJ_1_junction_aa', 'IR_VDJ_2_junction_aa', 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_productive', 'IR_VJ_2_productive', 'IR_VDJ_1_productive', 'IR_VDJ_2_productive', 'IR_VJ_1_v_call', 'IR_VJ_2_v_call', 'IR_VDJ_1_v_call', 'IR_VDJ_2_v_call', 'receptor_type', 'receptor_subtype', 'chain_pairing', 'clone_id', 'c

In [6]:
# Basic preprocessing (if not already done)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [7]:
# Perform differential expression analysis for all clusters
sc.tl.rank_genes_groups(adata, 'refined_clusters', method='wilcoxon')

# Get results as a dataframe for all clusters
marker_genes = pd.DataFrame()
for group in adata.obs['refined_clusters'].unique():
    df = sc.get.rank_genes_groups_df(adata, group=group)
    df['cluster'] = group
    marker_genes = pd.concat([marker_genes, df])

# Sort by scores and save to CSV
marker_genes = marker_genes.sort_values(['cluster', 'scores'], ascending=[True, False])
marker_genes.to_csv('cluster_marker_genes.csv', index=False)

d:\anaconda3\envs\pyautogen\lib\site-packages\scanpy\tools\_rank_genes_groups.py:435: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
d:\anaconda3\envs\pyautogen\lib\site-packages\scanpy\tools\_rank_genes_groups.py:437: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
d:\anaconda3\envs\pyautogen\lib\site-packages\scanpy\tools\_rank_genes_groups.py:440: PerformanceWarning: DataFrame is highly fragmented.  This is 

In [ ]:
# Perform differential expression analysis
# Note: Replace 'cluster_column' with your actual column name containing cell groups
# and 'group1' and 'group2' with the groups you want to compare
de_genes = sc.tl.rank_genes_groups(adata, 'refined_clusters', groups=['group1'], reference='group2', method='wilcoxon')

# Get results as a dataframe
de_results = sc.get.rank_genes_groups_df(adata, group='group1')

# Display top differentially expressed genes
print(de_results.head())

In [1]:
def get_top_markers(df, n_genes=10, format_type=None):
    """
    Get top markers from either Seurat or Scanpy differential expression results.
    
    Args:
        df: Either a pandas DataFrame (Seurat format) or dictionary (Scanpy format)
        n_genes: Number of top genes to return per cluster
        format_type: Either 'seurat', 'scanpy', or None (auto-detect)
    
    Returns:
        pandas DataFrame with cluster and marker columns
    """
    # Auto-detect format if not specified
    if format_type is None:
        if 'names' in df and 'scores' in df:
            format_type = 'scanpy'
        else:
            format_type = 'seurat'
    
    if format_type == 'scanpy':
        # Process Scanpy format
        clusters = df['names'].dtype.names
        top_markers = []
        
        for cluster in clusters:
            # Get data for this cluster
            genes = df['names'][cluster][:n_genes]
            logfc = df['logfoldchanges'][cluster][:n_genes]
            pvals_adj = df['pvals_adj'][cluster][:n_genes]
            pcts = df['pcts'][cluster][:n_genes]  # Get percentage information
            
            # Filter for significant upregulated genes with PCT threshold
            mask = (pvals_adj < 0.05) & (logfc > 0.25) & (pcts >= 0.1)  # Add PCT filter
            valid_genes = genes[mask]
            
            if len(valid_genes) > 0:
                # Join genes with commas
                markers = ','.join(valid_genes)
                top_markers.append({
                    'cluster': cluster,
                    'markers': markers
                })
        
        return pd.DataFrame(top_markers)
    
    else:  # Seurat format
        # Filter by adjusted p-value, positive log2FC, and PCT
        df_filtered = df[
            (df['p_val_adj'] < 0.05) & 
            (df['avg_log2FC'] > 0.25) &
            ((df['pct.1'] >=0.1) | (df['pct.2'] >=0.1))  # Add PCT filter
        ].copy()
        
        # Sort within each cluster by avg_log2FC and get top n genes
        top_markers = []
        
        for cluster in df_filtered['cluster'].unique():
            cluster_data = df_filtered[df_filtered['cluster'] == cluster]
            # Sort by avg_log2FC in descending order and take top n
            top_n = (cluster_data
                    .sort_values('avg_log2FC', ascending=False)
                    .head(n_genes))
            
            # Verify sorting worked correctly
            if not top_n.empty and not top_n['avg_log2FC'].is_monotonic_decreasing:
                print(f"Warning: Sorting issue detected for cluster {cluster}")
            
            top_markers.append(top_n)
        
        # Combine all results
        if top_markers:
            top_markers = pd.concat(top_markers, ignore_index=True)
            
            # Create markers column by concatenating genes in order
            result = (top_markers
                     .groupby('cluster',observed=True)
                     .agg({'gene': lambda x: ','.join(x)})
                     .rename(columns={'gene': 'markers'})
                     .reset_index())
            
            return result
        else:
            return pd.DataFrame(columns=['cluster', 'markers'])    


#marker is the input csv directory


df = pd.read_csv(marker)

# If dataframe has only two columns, assume it's already processed
if len(df.columns) == 2:
    print("Using input dataframe directly as it appears to be pre-processed (2 columns)")
else:  # if it is not preprocessed, use the get_top_markers function to process it
    print("Processing input dataframe to get top markers")
    df = get_top_markers(df, n_genes=n_genes)

# If celltype_column is not specified, use the first column
if celltype_column is None:
    celltype_column = df.columns[0]

# If gene_column_name is not specified, use the second column
if gene_column_name is None:
    gene_column_name = df.columns[1]

NameError: name 'marker' is not defined